<a href="https://colab.research.google.com/github/rainson/colab_nbs/blob/main/ytd2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Basic functions and imports


In [ ]:
!pip install pytube
!pip install youtube_transcript_api
!pip install vtt_to_srt3

In [2]:
from pytube import YouTube, Playlist
from youtube_transcript_api import YouTubeTranscriptApi
import json
import requests
from lxml import html
from vtt_to_srt import vtt_to_srt

# from vtt_to_srt import vtts_to_srt
from youtube_transcript_api.formatters import JSONFormatter
from youtube_transcript_api.formatters import TextFormatter
from youtube_transcript_api.formatters import WebVTTFormatter

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

def download(yturl,out_dir,idx=None):
  formatter = WebVTTFormatter()
  yt = YouTube(yturl)
  yt.streams.filter(file_extension="mp4")
  title = yt.title
  # title=" ".join([s.capitalize() for s in title.split(" ")])
  if idx is not None:
    title="{}".format(idx)+title
  print("start {}".format(title))
  try:
    yt.streams.get_by_itag(22).download(output_path=out_dir,filename=title)
  except Exception:
    yt.streams.get_by_itag(18).download(output_path=out_dir,filename=title)

  try:
    caption = yt.captions["en"]
    caption.download(title=title,output_path=out_dir)
  except Exception as ex:
    print(f"no en subtitle in {yt.title} finished")
  try:
    caption = yt.captions["a.en"]
    caption.download(title=title,output_path=out_dir)
  except Exception as ex:
    print(f"no a.en subtitle in {yt.title} finished")
  

  try:
    transcript_list = YouTubeTranscriptApi.list_transcripts(yt.video_id)
    transcript_list

    transcript = transcript_list.find_transcript(['en'])
    translated_transcript = transcript.translate('zh-Hans')
    zh_trans = translated_transcript.fetch()
    en_trans = transcript.fetch()

    zh_trans = formatter.format_transcript(zh_trans)
    en_trans=formatter.format_transcript(en_trans)

    fname = out_dir+"/{}(en).vtt".format(title)
    with open(fname, "w") as fh:
        fh.write(en_trans)
    # vtt_to_srt(fname)
    fname = out_dir+"/{}(zh).vtt".format(title)
    with open(fname, "w") as fh:
        fh.write(zh_trans)
    # vtt_to_srt(fname)
    fname =  out_dir+"/{}(zhen).vtt".format(title)
    with open(fname, "w") as fh:
        fh.write(zh_trans+en_trans)
    # vtt_to_srt(fname)
  except Exception:
    pass
  print("finish {}".format(title))

# Find urls in one page


In [ ]:
import re
# page_url = "https://www.youtube.com/channel/UCAKQnU09rjmdVRb3iAogCiQ/videos" #Atomic scale Jose Lado
# page_url = "https://www.youtube.com/c/ILSIITR/videos" #MOU Institute Lecture Series
page_url = "https://www.youtube.com/channel/UCM06KiUOw1vRrmvD8U274Ww/videos" #Harvard CMSA quantum-matter-seminar
page = requests.get(page_url)
webpage = html.fromstring(page.content)

video_ids = re.findall(r"watch\?v=(\S{11})", page.content.decode())
print(video_ids)
urls = ["https://www.youtube.com/watch?v={}".format(vid) for vid in video_ids]
urls = urls[::-1] # reversed for some special case

out_dir = "/content/drive/Shareddrives/OBA/HarvardCMSA"

['uYmA14y5gro', 'cK186hBlMms', 'WjfaMl6-Tek', 'CRlsuQKnGLk', 'R7rG5aRaWcE', 'RNvMDKnevlE', 'fyyPvGl7Zj0', 'JML1hlheVKo', 'edD-omyVSMM', 'ISOXhkzdpyc', 'UOaCmF1JEeU', '5een8U5cfbI', 'W2ArMz3J2gI', 'UnUQePiSGKo', 'x24KlhARrKA', 'QfTVaUjn_yg', 'wtaC0tqXZMU', 'T97wv7767AE', 'aVhHTmOUxHQ', 'AxWWOrM53bI', 'mYKlMyTWOhc', 'jGuzgdmtOhQ', 'vo8Yoqh9gb0', '068iZvB2b-Y', '_-nAVEtX4nE', 'mAYnLdi3vs4', 'tTpbHL7cZ5Y', '7Q8GvzJT-aM', 'ZoGfKvCxV6w', 'bERRfF7n9Hg']


# Urls in playlist


In [4]:
# plisturl = "https://www.youtube.com/playlist?list=PL7OOhDqj9FBuK3han8ipaKnz2SaGUjvGx"# topo cm
# plisturl="https://www.youtube.com/playlist?list=PLygH4pJ_4EhrwydaPvccVQYt1W6ytaYbU" # tms19
# plisturl = "https://www.youtube.com/playlist?list=PLygH4pJ_4EhogZda-GSFOBhlr2lSHn_p2" #tms18
# plisturl = "https://www.youtube.com/playlist?list=PLVqoFIN-64LnUQxbX2FeIGYQgX_VTWm-m" #tms17
# plisturl = "https://www.youtube.com/playlist?list=PLLv6hTkOwhEl1itfLvR-IcvOFt-aCxSzy" tms16
# plisturl = "https://www.youtube.com/playlist?list=PLE9Qrf4CJnRHxa_OwN7xc2bXacnWzCy1x" #opensys1
# plisturl = "https://www.youtube.com/playlist?list=PLd9T7y2ABtzJuN2YgujzWZ6fL-llEyuUd" #opensys2
# plisturl = "https://www.youtube.com/watch?v=g2RwzmG0spo&list=PL2IOBr0XELQSaB14lQgdFNvKHzs0W15hh" #UTphysics2020
# plisturl = "https://www.youtube.com/watch?v=BSV8zKbKc_Y&list=PL2IOBr0XELQQUjDBrh2tvSHhjHu9PHFYF"
# plisturl = "https://www.youtube.com/playlist?list=PLZee5Mirzz8ULGEdfuWM0DaoM-Yqtz9r8" #pcmp2014
# plisturl = "https://www.youtube.com/playlist?list=PLZee5Mirzz8WkK2EtlY1De7msJ6x9vP3T" #pcmp2015
# plisturl = "https://www.youtube.com/playlist?list=PLZee5Mirzz8UaZUDJZmh1BCV3xIjEPNEw" #pcmp2016
# plisturl = "https://www.youtube.com/playlist?list=PL04QVxpjcnjgmUoD6tlO_0YT1k5GYgQWi" #flucations in non equlibrium 
# plisturl = "https://www.youtube.com/playlist?list=PLgLljLUllyOcO6g2hoVlPiIKYYdpnlrwP" #IQUIST Seminars need reversed 更新中 https://iquist.illinois.edu/events/virtual-seminar-series
# plisturl = "https://www.youtube.com/playlist?list=PL0NRmB0fnLJQAnYwkpt9PN2PBKx4rvdup" #Quantum Matter Program Havard
# plisturl = "https://www.youtube.com/playlist?list=PLqJ4D_Db7W_qBCNdmJ2QaoenrXWCs82v0" #vsf speaker's corner need reversed
# plisturl="https://www.youtube.com/playlist?list=PLVk0rija2HWzNDNmoScO8HyxgsraJp2nh" #Online Spintronics Seminar 更新中https://www.spintalks.org/home
# plisturl = "https://www.youtube.com/playlist?list=PLGntAYRT8AVkDw1cPtAvp70rrl3nAXdOG" # vasp1
# plisturl = "https://www.youtube.com/playlist?list=PLGntAYRT8AVk9EV5Fm8qRboc3RCReTqGm" #Quanutmnerd Hands on qe 2019
plisturl = "https://www.youtube.com/playlist?list=PLZee5Mirzz8WI_Un45uDkTgFjhqg7LI0f" #2020 Princeton Summer School on Condensed Matter Physics (PSSCMP) PCCM


plist = Playlist(plisturl)
urls = plist.video_urls
# urls = urls[::-1] # reversed for some special case
# out_dir = "/content/drive/Shareddrives/OBA/IQUISTSeminars"
# out_dir = "/content/drive/Shareddrives/OBA/HarvardCMSA/QuantumMatterProgram"
# out_dir = "/content/drive/Shareddrives/OBA/VSF/SpeakerCorner"
# out_dir = "/content/drive/Shareddrives/OBA/OnlineSpintronicsSeminar"
# out_dir = "/content/drive/Shareddrives/OBA/vasp/vasp1"
# out_dir = "/content/drive/Shareddrives/OBA/QE/Hands-onQuantumEspressoTutorial2019"
out_dir = "/content/drive/Shareddrives/OBA/PCCM/PSSCMP2020"
print(len(urls))

14


# Url manully


In [ ]:
## non hermitian 2021
# urls = ["https://www.youtube.com/watch?v=MiyTEVq8BJ0",
#         "https://www.youtube.com/watch?v=wor5vdmBH_E",
#         "https://www.youtube.com/watch?v=s3qXZbU7ivk",
#         "https://www.youtube.com/watch?v=ON8hMzgLEHo",
#         "https://www.youtube.com/watch?v=gtC1LBLYrPk",
#         "https://www.youtube.com/watch?v=b1qrdafHMp0",
#         "https://www.youtube.com/watch?v=xcIFxekzy3Q",
#         "https://www.youtube.com/watch?v=BnIuitcOhmE",
#         "https://www.youtube.com/watch?v=1zV49ovoGN8",
#         "https://www.youtube.com/watch?v=Tpm0rSPw_9I"
#         ] 

# Download task

In [ ]:
for idx, yturl in enumerate(urls): #plist.video_urls
  try:
    download(yturl, out_dir,idx=None)
  except Exception as ex:
    print(f"exception {ex}")
vtt_to_srt.vtts_to_srt(out_dir, rec = True)

start Prof. Leslie Schoop: "Using Chemical Logic for the Design of New Quantum Materials"
no en subtitle in Prof. Leslie Schoop: "Using Chemical Logic for the Design of New Quantum Materials" finished
finish Prof. Leslie Schoop: "Using Chemical Logic for the Design of New Quantum Materials"
start Prof. Leon Balents: "Spin Liquids" (Lecture 1 of 2)


In [ ]:
!cd /content/drive/Shareddrives/OBA/PCCM/PSSCMP2020; tar -cvf /content/drive/Shareddrives/OBA/PSSCMP2020.tar ./*


In [ ]:
!rm -rf /content/drive/Shareddrives/OBA/HarvardCMSA/*